# 2025 09 25 - OpenAI API

# 삼성전자 주가분석

In [61]:
from pykrx import stock
from openai import OpenAI
import pandas as pd
import os
import re
from datetime import datetime, timedelta
from dotenv import load_dotenv
import gradio as gr

# 환경변수 로드
load_dotenv(override=True)

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
default_model = os.getenv("OPENAI_DEFAULT_MODEL")

# 1. ticekr, 30 - 티커 회사, 현재를 기준으로 -30일간 (8월26~9월25일)
# 2. 해당회사의 30일간의 주식 내용을 가져오기 - get_stock_price(ticker, days)
#  df = 가져온 데이터(30일간의 데이터)  return
# 3. 리턴 받은 데이터 (df)를 분석하기 위해서 LLM을 호출(prompt) -> analyze_stock(df, company, days)

# OHLCV -> df(open, high, low, close, volume)

In [15]:
def get_stock_price(ticker, days=30):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    #OHLCV
    df = stock.get_market_ohlcv(start_date.strftime("%Y%m%d"),
                                end_date.strftime("%Y%m%d"),
                                ticker)
    return df

In [17]:
def analyze_stock(df, company="삼성전자", days=30):
    if df.empty:
        return "분석할 데이터가 없습니다"
    
    latest_price = df['종가'].iloc[-1]
    highest_price = df['종가'].max()
    lowest_price = df['종가'].min()
    average_price = df['종가'].mean()

    analysis = ( f"최근 {days}일 동안 삼성전자의 종가 데이터를 분석한 결과는 다음과 같습니다\n" f"Latest Price: {latest_price}\n" f"Highest Price in the period: {highest_price}\n" f"Lowest Price in the period: {lowest_price}\n" f"Average Price in the period: {average_price}" f"이 데이터를 기반으로 간단한 분석과 전망을 제공해주세요" )

    ret = client.chat.completions.create(

        model=default_model,
        messages=[{"role":"system", "content":"너는 주식분석 전문가야"}, {"role":"user", "content":analysis}],
        temperature=0,
        max_tokens=500
    )

    return ret.choices[0].message.content.strip()

In [18]:
ticker = "005930" # 삼성전자

df = get_stock_price(ticker, days=30)
display(df)

print(analyze_stock(df))


,시가,고가,저가,종가,거래량,등락률
날짜,,,,,,
2025-08-26,70800,71100,70300,70300,14712519,-1.678322
2025-08-27,70100,70900,69800,70600,10523405,0.426743
2025-08-28,70100,70400,69600,69600,11578024,-1.416431
2025-08-29,70100,70500,69700,69700,11682267,0.143678
2025-09-01,68400,68600,67500,67600,12002343,-3.012912
2025-09-02,67800,69500,67800,69100,10604028,2.218935
2025-09-03,69200,69800,68800,69800,10283009,1.013025
2025-09-04,69500,70100,69300,70100,12284414,0.429799
2025-09-05,70300,70400,69500,69500,11526724,-0.855920


삼성전자의 최근 30일 종가 데이터를 기반으로 한 분석은 다음과 같습니다.

### 1. 가격 변동성
- **최고가**: 85,900원
- **최저가**: 67,600원
- **평균가**: 약 74,917원

최고가와 최저가의 차이가 상당히 크기 때문에, 이 기간 동안 삼성전자의 주가는 높은 변동성을 보였음을 알 수 있습니다. 이는 시장의 불확실성이나 외부 요인(예: 글로벌 경제 상황, 반도체 수요 변화 등)에 영향을 받을 수 있습니다.

### 2. 최근 가격 동향
현재 가격이 85,900원으로, 최근 30일 동안의 최고가와 동일합니다. 이는 주가가 최근에 강한 상승세를 보였음을 나타내며, 투자자들의 긍정적인 심리가 반영된 결과일 수 있습니다. 

### 3. 평균가와의 비교
현재 가격이 평균가(약 74,917원)보다 상당히 높은 수준에 위치하고 있습니다. 이는 주가가 평균적으로는 상승세를 보이고 있으며, 시장에서 긍정적인 평가를 받고 있다는 신호일 수 있습니다.

### 4. 전망
- **긍정적 시나리오**: 현재의 상승세가 지속된다면, 추가적인 상승이 가능할 수 있습니다. 특히, 반도체 산업의 회복이나 새로운 제품 출시 등의 호재가 있다면 주가는 더욱 상승할 가능성이 있습니다.
  
- **부정적 시나리오**: 반면, 주가가 이미 최고가에 도달한 만큼, 조정이 올 가능성도 존재합니다. 특히, 글로벌 경제 불확실성이나 실적 발표 후 부정적인 반응이 있을 경우 주가가 하락할 수 있습니다.

### 결론
삼성전자의 주가는 최근 긍정적인 흐름을 보이고 있으나, 높은 변동성을 감안할 때 투자 시 주의가 필요합니다. 향후 시장 동향과 기업 실적을 면밀히 관찰하는 것이 중요합니다.


In [10]:
COMPANY_TICKERS = {
    "삼성전자": "005930",
    "SK하이닉스": "000660", 
    "NAVER": "035420",
    "네이버": "035420",
    "카카오": "035720",
    "LG화학": "051910",
    "삼성SDI": "006400",
    "현대차": "005380",
    "기아": "000270",
    "POSCO홀딩스": "005490",
    "포스코홀딩스": "005490",
    "LG전자": "066570",
    "한국전력": "015760",
    "신한지주": "055550",
    "KB금융": "105560",
    "셀트리온": "068270",
    "삼성바이오로직스": "207940",
    "LG에너지솔루션": "373220",
    "SK텔레콤": "017670",
    "KT&G": "033780",
    "한화에어로스페이스": "012450",
    "두산에너빌리티": "034020",
    "HD현대중공업": "329180",
    "삼성물산": "028260",
    "CJ제일제당": "097950",
    "아모레퍼시픽": "090430",
    "LG생활건강": "051900",
    "현대건설": "000720",
    "GS": "078930",
    "롯데케미칼": "011170",
    "현대모비스": "012330"
}

def find_ticker(company_name):
    """회사명으로 티커 찾기"""
    company_name = company_name.strip()
    
    # 직접 매칭
    if company_name in COMPANY_TICKERS:
        return COMPANY_TICKERS[company_name]
    
    # 부분 매칭
    for name, ticker in COMPANY_TICKERS.items():
        if company_name in name or name in company_name:
            return ticker
    
    # 6자리 숫자면 티커로 간주
    if re.match(r'^\d{6}$', company_name):
        return company_name
    
    return None

In [11]:
def parse_date(date_str):
    """날짜 문자열을 파싱"""
    try:
        # YYYY-MM-DD 형식
        if re.match(r'\d{4}-\d{2}-\d{2}', date_str):
            return datetime.strptime(date_str, '%Y-%m-%d')
        # YYYYMMDD 형식
        elif re.match(r'\d{8}', date_str):
            return datetime.strptime(date_str, '%Y%m%d')
        # YYYY.MM.DD 형식
        elif re.match(r'\d{4}\.\d{2}\.\d{2}', date_str):
            return datetime.strptime(date_str, '%Y.%m.%d')
        # YYYY/MM/DD 형식
        elif re.match(r'\d{4}/\d{2}/\d{2}', date_str):
            return datetime.strptime(date_str, '%Y/%m/%d')
    except:
        pass
    return None

In [12]:
def extract_analysis_params(message):    
    params = {}
    
    # 회사명 추출 (첫 번째 단어나 따옴표로 감싸진 부분)
    company_match = re.search(r'["\']([^"\']+)["\']', message)
    if company_match:
        params['company'] = company_match.group(1)
    else:
        # 첫 번째 단어를 회사명으로 간주
        words = message.split()
        if words:
            params['company'] = words[0]
    
    # 날짜 추출
    date_patterns = [
        r'\d{4}-\d{2}-\d{2}',
        r'\d{8}',
        r'\d{4}\.\d{2}\.\d{2}',
        r'\d{4}/\d{2}/\d{2}'
    ]
    
    dates = []
    for pattern in date_patterns:
        dates.extend(re.findall(pattern, message))
    
    if len(dates) >= 2:
        params['start_date'] = parse_date(dates[0])
        params['end_date'] = parse_date(dates[1])
    elif len(dates) == 1:
        params['end_date'] = parse_date(dates[0])
        params['start_date'] = params['end_date'] - timedelta(days=30)
    
    # 기간 추출 (30일, 3개월 등)
    period_match = re.search(r'(\d+)(일|개월|달|년)', message)
    if period_match and 'start_date' not in params:
        num = int(period_match.group(1))
        unit = period_match.group(2)
        
        end_date = datetime.now()
        if unit == '일':
            start_date = end_date - timedelta(days=num)
        elif unit in ['개월', '달']:
            start_date = end_date - timedelta(days=num*30)
        elif unit == '년':
            start_date = end_date - timedelta(days=num*365)
        
        params['start_date'] = start_date
        params['end_date'] = end_date
    
    return params

In [13]:
def get_stock_price(ticker, start_date, end_date):
    """주식 데이터 가져오기"""
    try:
        df = stock.get_market_ohlcv_by_date(
            start_date.strftime('%Y%m%d'), 
            end_date.strftime('%Y%m%d'), 
            ticker
        )
        return df
    except Exception as e:
        print(f"Error fetching stock data: {e}")
        return pd.DataFrame()

def analyze_stock_data(df, company_name, start_date, end_date):
    """주식 데이터 분석"""
    if df.empty:
        return f"{company_name}의 주식 데이터를 찾을 수 없습니다."
    
    latest_price = df['종가'].iloc[-1]
    highest_price = df['고가'].max()
    lowest_price = df['저가'].min()
    average_price = df['종가'].mean()
    
    # 가격 변동률 계산
    first_price = df['종가'].iloc[0]
    price_change = latest_price - first_price
    price_change_pct = (price_change / first_price) * 100
    
    # 거래량 분석
    avg_volume = df['거래량'].mean()
    latest_volume = df['거래량'].iloc[-1]
    
    period_str = f"{start_date.strftime('%Y-%m-%d')}부터 {end_date.strftime('%Y-%m-%d')}까지"
    
    analysis = f"""
{company_name}의 {period_str} 주식 분석 데이터:

가격 정보:
- 최신 종가: {latest_price:,}원
- 기간 내 최고가: {highest_price:,}원
- 기간 내 최저가: {lowest_price:,}원
- 평균 가격: {average_price:,.0f}원

수익률 분석:
- 기간 수익률: {price_change:+,.0f}원 ({price_change_pct:+.2f}%)

거래량 정보:
- 평균 거래량: {avg_volume:,.0f}주
- 최근 거래량: {latest_volume:,.0f}주

위 데이터를 바탕으로 {company_name}의 주식에 대한 전문적인 분석과 투자 관점에서의 의견을 제공해주세요. 
기술적 분석, 리스크 요인, 향후 전망 등을 포함해서 설명해주세요.
"""
    
    return analysis


In [14]:
def respond(message, history):
    try:
        # 1. 주식 분석 요청인지 확인
        stock_keywords = ['주식', '분석', '종목', '투자', '차트', '가격', '주가']
        is_stock_query = any(keyword in message.lower() for keyword in stock_keywords)
        
        if is_stock_query:
            # 2. 분석 파라미터 추출
            params = extract_analysis_params(message)
            
            if 'company' in params:
                company_name = params['company']
                ticker = find_ticker(company_name)
                
                if ticker:
                    # 기본 날짜 설정
                    end_date = params.get('end_date', datetime.now())
                    start_date = params.get('start_date', end_date - timedelta(days=30))
                    
                    # 주식 데이터 가져오기
                    df = get_stock_price(ticker, start_date, end_date)
                    
                    if not df.empty:
                        # 분석 요청 생성
                        analysis_prompt = analyze_stock_data(df, company_name, start_date, end_date)
                        
                        # 메시지 히스토리에 분석 데이터 추가
                        messages = [{"role": "system", "content": "당신은 주식 분석 전문가입니다. 제공된 데이터를 바탕으로 전문적이고 객관적인 분석을 제공하세요."}]
                        
                        for user, bot in history:
                            messages.append({"role": "user", "content": user})
                            messages.append({"role": "assistant", "content": bot})
                        
                        messages.append({"role": "user", "content": analysis_prompt})
                    else:
                        messages = [{"role": "system", "content": "당신은 친절한 AI 챗봇입니다."}]
                        for user, bot in history:
                            messages.append({"role": "user", "content": user})
                            messages.append({"role": "assistant", "content": bot})
                        messages.append({"role": "user", 
                                        "content": f"{company_name}(티커: {ticker})의 주식 데이터를 찾을 수 없습니다. 다른 기업을 시도해보세요."})
                else:
                    messages = [{"role": "system", "content": "당신은 친절한 AI 챗봇입니다."}]
                    for user, bot in history:
                        messages.append({"role": "user", "content": user})
                        messages.append({"role": "assistant", "content": bot})
                    messages.append({"role": "user", 
                                    "content": f"'{company_name}'의 티커를 찾을 수 없습니다. 다음 형식으로 요청해주세요:\n예: '삼성전자 30일 분석' 또는 '네이버 2024-01-01 2024-01-31 분석'"})
            else:
                # 일반 주식 관련 질문
                messages = [{"role": "system", "content": "당신은 주식 투자 전문가입니다."}]
                for user, bot in history:
                    messages.append({"role": "user", "content": user})
                    messages.append({"role": "assistant", "content": bot})
                messages.append({"role": "user", "content": message})
        else:
            # 일반 채팅
            messages = [{"role": "system", "content": "당신은 친절한 AI 챗봇입니다."}]
            for user, bot in history:
                messages.append({"role": "user", "content": user})
                messages.append({"role": "assistant", "content": bot})
            messages.append({"role": "user", "content": message})
        
        # OpenAI API 호출 (스트리밍)
        response = client.chat.completions.create(
            model=default_model,
            messages=messages,
            temperature=0.7,
            stream=True,
        )
        
        response_stream = ""
        for chunk in response:
            if hasattr(chunk, 'choices') and len(chunk.choices) > 0:
                delta = chunk.choices[0].delta
                if hasattr(delta, 'content') and delta.content:
                    content = delta.content
                    response_stream += content
                    yield history + [(message, response_stream)], ""
    
    except Exception as e:
        error_msg = f"오류가 발생했습니다: {str(e)}"
        yield history + [(message, error_msg)], ""

# 캠핑장 키워드 찾기

In [25]:
camping_api = os.getenv("GO_CAMPING_API_KEY")
print(camping_api)

3d583a9839c8fee7ab801e0caa33c90104a20066db91063b69f2accaf3b26884


In [32]:
keyword = '계곡'
url = f'http://apis.data.go.kr/B551011/GoCamping/searchList?serviceKey={camping_api}&MobileOS=ETC&MobileApp=AppTest&keyword={keyword}&_type=json'
print(url)

http://apis.data.go.kr/B551011/GoCamping/searchList?serviceKey=3d583a9839c8fee7ab801e0caa33c90104a20066db91063b69f2accaf3b26884&MobileOS=ETC&MobileApp=AppTest&keyword=계곡&_type=json


In [36]:
# 키워드 추출
question = "계곡이 있는 캠핑장을 추천해줘"
question = "가족이 주말에 함께 여행할 캠핑장 추천해줘"
response = client.chat.completions.create(
    model=default_model,
    messages=[
        {"role":"system", "content":"다음 질문에서 가장 중요한 키워드 단어 하나만 뽑아줘"},
        {"role":"user", "content":question}],
        temperature=0,
        max_tokens=256

)
keyword = response.choices[0].message.content.strip()
print(keyword)

캠핑장


In [37]:
url = f'http://apis.data.go.kr/B551011/GoCamping/searchList?serviceKey={camping_api}&MobileOS=ETC&MobileApp=AppTest&keyword={keyword}&_type=json'
print(url)

http://apis.data.go.kr/B551011/GoCamping/searchList?serviceKey=3d583a9839c8fee7ab801e0caa33c90104a20066db91063b69f2accaf3b26884&MobileOS=ETC&MobileApp=AppTest&keyword=캠핑장&_type=json


In [42]:
import requests

res = requests.get(url)
if res.status_code == 200:
    data = res.json()
else:
    print(f"Error : {res.status_code}")

print(data)
result = data['response']['body']['items']['item']

context = ""
for item in result:
    context += item['facltNm']+":"+item['induty']+':'+item['doNm']+':'+item['intro']+'\n'

print(context)


{'response': {'header': {'resultCode': '0000', 'resultMsg': 'OK'}, 'body': {'items': {'item': [{'contentId': '8073', 'facltNm': '파인트리캠핑장', 'lineIntro': '개미들마을 초입 개울가에 자리한 대형 캠핑장', 'intro': '정선에서 사북가는길 개미들마을초입에 자리잡고 있다. 캠핑장 바로 앞으로 제법 크고 맑은 개울이 흐르고 이곳에서 물놀이와 민물낚시도 할 수 있다. 사방이 초록 초록한 산속에 위치한 정선 파인트리 캠핑장은 자연을 느끼며 힐링 할 수 있는 캠핑장으로 캠퍼들에게 인기다. 수영장과 트램펄린이 있어 아이들과 함께 하기에도 좋으며, 산책하기 좋은 산책로와, 가까운 하천에서는 다슬기를 잡거나 가벼운 물놀이도 즐길 수 있다. 사이트에 나무 그늘이 적어 여름에는 타프가 꼭 필요하다. 시설이 깨끗하고 깔끔하게 관리가 잘 되어 있다.', 'allar': '0', 'insrncAt': 'Y', 'trsagntNo': '2021-1', 'bizrno': '466-38-00746', 'facltDivNm': '민간', 'mangeDivNm': '직영', 'mgcDiv': '공석률', 'manageSttus': '운영', 'hvofBgnde': '', 'hvofEnddle': '', 'featureNm': '', 'induty': '일반야영장', 'lctCl': '산,숲,계곡', 'doNm': '강원도', 'sigunguNm': '정선군', 'zipcode': '26143', 'addr1': '강원 정선군 남면 광락로 349-30', 'addr2': '', 'mapX': '128.693720520755', 'mapY': '37.3161381592256', 'direction': '', 'tel': '010-8879-2470', 'homepage': 'https://sites.google.com/view/pinetrees7777/%ED%99%88', 're

In [ ]:
response = client.chat.completions.create(
    model=default_model,
    messages=[
        {"role":"system", "content":f"주어진 문단을 보고 다음 질문에 답해줘:{question}"},
        {"role":"user", "content":context}],
        temperature=0,
        max_tokens=256ㅠㅠㅠ

)
keyword = response.choices[0].message.content.strip()
print(keyword)

가족이 주말에 함께 여행할 캠핑장으로 다음을 추천합니다:

1. **파인트리캠핑장 (강원도 정선)**: 맑은 개울이 흐르고 물놀이와 민물낚시를 즐길 수 있는 곳으로, 아이들과 함께하기 좋은 수영장과 트램펄린이 있어 가족 단위 방문에 적합합니다.

2. **꿈꾸는 캠핑장 (전라남도 곡성)**: 다양한 나무들로 둘러싸인 숲 속에 위치하며, 할로윈 시즌에는 이색적인 이벤트가 있어 아이들이 즐길 수 있는 특별한 경험을 제공합니다.

3. **꿈꾸는 농부 오토캠핑장 (대전광역시)**: 청량감 가득한 환경에서 물놀이를 즐길 수 있으며, 인근에 유명 관광지가 많아 연계 관광이 용이합니다. 반려동물도 소형견에 한해 출입이 가능해 가족과 함께하는 캠핑에 좋습니다.

4. **꿈꾸는바다캠핑장 (충청남도 태안)**: 태안 앞바다를 바라보며


# functions / tools

In [74]:
city = '서울'
response = client.chat.completions.create (
    model = default_model,
    messages=[{"role":"user", "content":f"{city}의 오늘 날씨 알려줘"}]
)
print(response.choices[0].message.content.strip())

죄송하지만, 실시간 날씨 정보를 제공할 수는 없습니다. 오늘 서울의 날씨를 확인하시려면 기상청 웹사이트나 날씨 앱을 이용해보시기를 권장합니다.


In [75]:
def get_weather(city):
    weather_data = {
        "서울" : "맑음, 25",
        "부산" : "비, 22도",
        "대구" : "흐림, 24도",
        "인천" : "맑음, 28도"
    }

    return weather_data.get(city, "정보없음")
def get_time(city):
    import datetime
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"{city}의 현재 시간은 {current_time}입니다"

In [76]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "오늘의 날씨를 알려주는 함수",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "도시이름",
                    },
                },
                "required": ["city"],
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "현재 시간을 알려주는 함수",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "도시이름",
                    },
                },
                "required": ["city"],
            }
        }
    }
]

In [77]:
city = '서울'
response = client.chat.completions.create(
    model=default_model,
    messages=[{"role":"user", "content":f"{city}의 오늘 날씨 알려줘"}],
    tools=tools,
    tool_choice="auto"
)
print(response.choices[0].message.content)

None


In [78]:
print(response.choices[0].message.tool_calls[0].function.name)

get_weather


In [79]:
# 사용가능한 함수들을 딕셔너리로 매핑
available_functions = {
    "get_weather": get_weather,
    "get_time": get_time
}
import json
if response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]
    print(f'{tool_call.function.name}')
    print(f'{tool_call.function.arguments}')
    # 동적으로 함수 호출
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    if function_name in available_functions:
        function_to_call = available_functions[function_name]
        weather_info = function_to_call(**function_args)
        print(weather_info)
    else:
        print(f"함수 {function_name}을 찾을 수 없습니다.")

get_weather
{"city":"서울"}
맑음, 25
